In [46]:
# explicit tutorial on the encoding
# what gates do encoding and detection
# the known solutions are the 5-wave and parallel 3-wave
# (we don't know if there exists others yet)
# but we need to first be able to simulate the faults and effective increases in lifetimes
# if that the qubits had lifetimes and errro channels
import numpy as np
from IPython.display import display, Math

In [47]:
from quantum_logical.conversiongain import QubitInteraction

# TODO refactor to conversion gain
# we should be able to return the equation from the class
H = QubitInteraction(gc=np.pi / 2, gg=0, transmon_levels=2)
display(Math(str(H)))
U = H.construct_U(t=1.0)
print(U)

<IPython.core.display.Math object>

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


Quick introduction using standard phase repetition code
\begin{align}
|L_0\rangle = |+++\rangle\\
|L_1\rangle = |---\rangle
\end{align}

such that $|\psi\rangle = \alpha |L_0\rangle + \beta |L_1\rangle$

In [48]:
from quantum_logical.basis import PhaseReptition

encoding = PhaseReptition()

In [49]:
# to initialize the encoding:
encoding.initialize()

AttributeError: 'PhaseReptition' object has no attribute 'initialize'

To create logical operators use the following transformation:
\begin{align*}
A = |L_0\rangle\langle0| + |L_1\rangle\langle1|\\
U' = A U A^\dagger
\end{align*}
,where U is unitary that acts on the $|0\rangle$ and $|1\rangle$ states.

In [53]:
from qiskit import QuantumCircuit
from qiskit.quantum_info.operators import Operator

qc = QuantumCircuit(3)
qc.cx(0, 1)
qc.cx(0, 2)
Operator(qc).data.shape

(8, 8)

In [50]:
encoding.logical_basis.transform_operator

Quantum object: dims = [[2, 2, 2], [2]], shape = (8, 2), type = oper, isherm = False
Qobj data =
[[ 0.35355339  0.35355339]
 [ 0.35355339 -0.35355339]
 [ 0.35355339 -0.35355339]
 [ 0.35355339  0.35355339]
 [ 0.35355339 -0.35355339]
 [ 0.35355339  0.35355339]
 [ 0.35355339  0.35355339]
 [ 0.35355339 -0.35355339]]

Next, 

In [51]:
from quantum_logical.basis import SNAILConcatWithAncilla

encoding = SNAILConcatWithAncilla()
encoding.logical_basis.transform_operator

Quantum object: dims = [[3, 3, 3], [2]], shape = (27, 2), type = oper, isherm = False
Qobj data =
[[ 0.35355339  0.35355339]
 [ 0.          0.        ]
 [ 0.35355339 -0.35355339]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.35355339 -0.35355339]
 [ 0.          0.        ]
 [ 0.35355339  0.35355339]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.35355339 -0.35355339]
 [ 0.          0.        ]
 [ 0.35355339  0.35355339]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.35355339  0.35355339]
 [ 0.          0.        ]
 [ 0.35355339 -0.35355339]]